In [53]:
import pandas as pd
import numpy as np
import warnings
from sklearn.manifold import TSNE
import plotly.express as px
from helpers import load_data, get_embedding
import tqdm
import plotly.graph_objects as go
import json
import googlemaps
import os
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

DATA_PATH = 'data/'

USE_CITIES = False # If False, we use countries

# Load the data
loaded_data = load_data(DATA_PATH)

# We set all needed variables
character_metadata = loaded_data['character_metadata']
movie_metadata = loaded_data['movie_metadata']
plot_summaries = loaded_data['plot_summaries']
embeddings = loaded_data['embeddings']
combined_plot_summaries = loaded_data['combined_plot_summaries']
city_country_analysis = loaded_data['city_country_analysis']
cities = city_country_analysis['cities']
countries = city_country_analysis['countries']
cities_movies = city_country_analysis['cities_movies']
countries_movies = city_country_analysis['countries_movies']
embeddings_of_movies_in_cities = city_country_analysis['embeddings_of_movies_in_cities']
embeddings_of_movies_in_countries = city_country_analysis['embeddings_of_movies_in_countries']

In [45]:
def movie_id_to_name(wikipedia_movie_id):
    if wikipedia_movie_id is None:
        return None
    
    names = movie_metadata.loc[movie_metadata['Wikipedia movie ID'] == wikipedia_movie_id]['Movie name']

    if names.empty:
        return "Unknown"
    return names.values[0]

movie_names = combined_plot_summaries['Wikipedia movie ID'].apply(movie_id_to_name)

#### City representation from foreign countries 

In [46]:
# We load from movie_analysis.json and convert to df
with open('data/movie_analysis.json', 'r') as f:
    analysis = json.load(f)

In [47]:
# We convert the string in the values to a dict
for key, value in analysis.items():
    analysis[key] = json.loads(value)

In [48]:
def get_cities(one_analysis):
    if one_analysis is None:
        return []

    if 'cities' not in one_analysis:
        return []

    return one_analysis['cities']
    
cities = [get_cities(value) for key, value in analysis.items()]
cities = sorted(list(set([item for sublist in cities for item in sublist])))

In [49]:
def get_countries(one_analysis):
    if one_analysis is None:
        return []

    if 'countries' not in one_analysis:
        return []

    return one_analysis['countries']
    
countries = [get_countries(value) for key, value in analysis.items()]
countries = sorted(list(set([item for sublist in countries for item in sublist])))

In [54]:
gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

city_coordinates = {}
country_coordinates = {}


if USE_CITIES:
    for city in cities:
        geocode_result = gmaps.geocode(city)
        if geocode_result:
            lat = geocode_result[0]["geometry"]["location"]["lat"]
            lng = geocode_result[0]["geometry"]["location"]["lng"]
            city_coordinates[city] = (lat, lng)
        else:
            city_coordinates[city] = (None, None)
else:
    for country in countries:
        geocode_result = gmaps.geocode(country)
        if geocode_result:
            lat = geocode_result[0]["geometry"]["location"]["lat"]
            lng = geocode_result[0]["geometry"]["location"]["lng"]
            country_coordinates[country] = (lat, lng)
        else:
            country_coordinates[country] = (None, None)


In [55]:
import ast

# Function to extract country name (first country name)
def extract_country(country_string):
    if country_string:
        try:
            country_dict = ast.literal_eval(country_string)  # Convert string to dictionary
            if country_dict and isinstance(country_dict, dict):
                country_values = list(country_dict.values())
                if country_values:
                    return country_values[0]
        except (ValueError, SyntaxError):
            # Handle the case where the string is not a valid dictionary
            pass
    return None

# Apply the function to the DataFrame column
# Added a 'Country' column some values are None
movie_metadata['Country'] = movie_metadata['Movie countries (Freebase ID:name tuples)'].apply(extract_country)

In [126]:
### Standardize country names 

movie_metadata['Country'] = movie_metadata['Country'].replace('United States of America', 'United States')
movie_metadata['Country'] = movie_metadata['Country'].replace('England', 'United Kingdom')
movie_metadata['Country'] = movie_metadata['Country'].replace('Kingdom of Great Britain', 'United Kingdom')
movie_metadata['Country'] = movie_metadata['Country'].replace('German Democratic Republic', 'Germany')
movie_metadata['Country'] = movie_metadata['Country'].replace('Weimar Republic', 'Germany')
movie_metadata['Country'] = movie_metadata['Country'].replace('Serbia and Montenegro', 'Serbia')


In [57]:
cities_movies = {city: [] for city in cities}
countries_movies = {country: [] for country in countries}

for key, value in analysis.items():
    movie_cities = get_cities(value)
    movie_countries = get_countries(value)

    for city in movie_cities:
        if city in cities_movies:
            cities_movies[city].append(int(key))

    for country in movie_countries:
        if country in countries_movies:
            countries_movies[country].append(int(key))

In [58]:
# Remove all countries with less than 10 movies both from countries and countries_movies
countries = [country for country in countries if len(countries_movies[country]) >= 10]
countries_movies = {country: countries_movies[country] for country in countries}

In [59]:
# Remove all cities with less than 10 movies both from cities and cities_movies
cities = [city for city in cities if len(cities_movies[city]) >= 10]
cities_movies = {city: cities_movies[city] for city in cities}

#### Select a list of city we are interested in for poc 

check if there could be significant différences for a city violence représentation for example between two production movie countries

les pays qui font le plus de film sur Paris

majoritairement la vision qu ils en ont

que ce soit le genre du film ou autre et si ca differe de la norme ou du pays d'origine de la ville

In [106]:
## Cities of interest COI
city_of_interest = [
    'London'
]


In [107]:
# Extract list of movie IDS where the plot takes place in the city of intereset
coi_movies = {}
for city in city_of_interest:
    coi_movies[city] = cities_movies.get(city, [])

# Print the lists for each city
for city, movie_ids in coi_movies.items():
    print(f"{city}: {movie_ids}")

London: [32748561, 12938853, 19273689, 182737, 156632, 4596602, 707808, 28581630, 27970617, 28252265, 26113564, 5425205, 5150901, 4329971, 9313886, 484153, 33184757, 17404922, 11399740, 18698088, 23150767, 2821933, 12715927, 3737258, 6574607, 1161955, 6542034, 22361477, 4968653, 16652785, 949857, 1139763, 268833, 35705743, 29003237, 12270563, 887164, 16456979, 241603, 11761134, 3085944, 2745708, 247098, 9271078, 33942920, 210216, 80493, 1676093, 5047454, 2425302, 27485441, 3022675, 12583457, 28103735, 1659954, 865607, 300910, 19101562, 248601, 9951334, 24328800, 3027029, 19284029, 1623252, 19332675, 30785672, 3116779, 7377259, 179351, 1219214, 20453522, 11350779, 1061565, 10615842, 34760845, 1539714, 32784153, 10004055, 23766166, 2188611, 32803232, 13443734, 28149238, 1046559, 20762022, 22787999, 105434, 2249957, 13253372, 129312, 2478633, 36036304, 28352353, 24461292, 4380389, 17465733, 3822078, 1307956, 4073679, 23380332, 583273, 15052685, 3015828, 1428872, 1969580, 7287091, 217750, 

In [108]:
len(coi_movies['London'])

1176

In [63]:
cities[70:80]

['Copenhagen',
 'Cornwall',
 'Crystal Lake',
 'Cuba',
 'Dakar',
 'Dallas',
 'Damascus',
 'Darjeeling',
 'Delhi',
 'Denmark']

In [109]:
#### Non exhaustive list to exclude the production country of a city
cities_in_country ={
 'Argentina':['Buenos Aires',],
 'Australia':['Adelaide','Brisbane'],
 'Belgium':['Brussels'],
 'Canada':['Calgary',],
 'Egypt':['Alexandria','Cairo'],
 'France':['Paris','Cannes'],
 'Germany':['Berlin'],
 'Greece':['Athens'],
 'Holland':['Amsterdam'],
 'Hungary':['Budapest'],
 'India':['Bangalore','Bombay','Calcutta','Chandigarh','Chennai'],
 'Iraq':['Baghdad'],
 'Ireland':['Belfast'],
 'Lebanon':['Beirut'],
 'Mexico':['Acapulco'],
 'Morocco':['Casablanca'],
 'Serbia':['Belgrade'],
 'South Africa':['Cape Town'],
 'South Korea':['Busan'],
 'Spain':['Barcelona'],
 'Thailand':['Bangkok'],
 'United Kingdom':['Bath','Birmingham','Blackpool','Brighton','Bristol','Cambridge','London'],
 'United States':['Alabama','Alaska','Albuquerque','Arizona','Arkansas','Atlanta','Auckland','Austin','Albany', 
                  'Atlantic City','Bakersfield','Baltimore','Berkeley','Beverly Hills','Boston','Broadway',
                  'Bronx','Brooklyn','Buffalo','California','Cape Cod','Central Park','Charleston','Chicago',
                  'Cincinnati','New York'],
 'Venezuela':['Caracas'],
}

In [127]:
# Filter movie IDs based on the country in movie_metadata (takes out movies from the country of COI)
foreign_movies = {}
for city, movie_ids in coi_movies.items():
    filtered_movie_ids = []
    for movie_id in movie_ids:
        matching_movies = movie_metadata[movie_metadata['Wikipedia movie ID'] == movie_id]
        if not matching_movies.empty:
            movie_country = matching_movies.iloc[0]['Country']
            city_country = next((country for country, cities in cities_in_country.items() if city in cities), None)
            if movie_country is not None and movie_country != city_country:
                filtered_movie_ids.append(movie_id)
    foreign_movies[city] = filtered_movie_ids

# Print the filtered lists for each city
for city, movie_ids in foreign_movies.items():
    print(f"{city}: {movie_ids}")

London: [12938853, 707808, 27970617, 5425205, 4329971, 17404922, 11399740, 2821933, 12715927, 6542034, 4968653, 268833, 29003237, 16456979, 241603, 3085944, 9271078, 33942920, 80493, 5047454, 2425302, 27485441, 12583457, 1659954, 300910, 19101562, 9951334, 24328800, 19284029, 19332675, 179351, 1219214, 11350779, 10615842, 32784153, 10004055, 32803232, 13443734, 1046559, 22787999, 129312, 4380389, 17465733, 1307956, 23380332, 583273, 15052685, 26868913, 28413162, 4829230, 16150645, 33567266, 25433944, 25955274, 9092451, 13238459, 13702809, 3909180, 11507737, 18520285, 35981207, 10495987, 18702022, 183741, 15007082, 11925733, 488210, 14997446, 2952485, 34518277, 10133874, 18964723, 145892, 19208905, 448385, 8856844, 15896980, 1027060, 23160762, 4806533, 3116175, 982199, 18056742, 2509613, 1162528, 3682257, 556241, 17987664, 6149282, 204019, 6852733, 24486141, 6492028, 28853738, 13687674, 660343, 15861151, 4836072, 77223, 2750377, 14415759, 13938940, 2755048, 2297089, 9706751, 8049233, 45

In [128]:
# Create a dictionary to regroup movie IDs by country of production
regrouped_by_country = {}

for city, movie_ids in foreign_movies.items():
    for movie_id in movie_ids:
        matching_movies = movie_metadata[movie_metadata['Wikipedia movie ID'] == movie_id]
        if not matching_movies.empty:
            movie_country = matching_movies.iloc[0]['Country']
            if movie_country is not None:
                if movie_country not in regrouped_by_country:
                    regrouped_by_country[movie_country] = []
                regrouped_by_country[movie_country].append(movie_id)


In [129]:
# Create a dictionary to regroup movie IDs by city and country
regrouped_by_city_and_country = {}

for city, movie_ids in foreign_movies.items():
    city_dict = {}
    for movie_id in movie_ids:
        matching_movies = movie_metadata[movie_metadata['Wikipedia movie ID'] == movie_id]
        if not matching_movies.empty:
            movie_country = matching_movies.iloc[0]['Country']
            if movie_country is not None:
                if movie_country not in city_dict:
                    city_dict[movie_country] = []
                city_dict[movie_country].append(movie_id)
    regrouped_by_city_and_country[city] = city_dict


#### Could be used for a lot other variables than just number of movies or percentage of movies (with : genres / good &  evil  / .... ???)

In [122]:
len(foreign_movies['London'])

610

In [123]:
len(regrouped_by_city_and_country['London']['United States'])

401

In [124]:
(len(regrouped_by_city_and_country['London']['United States'])/len(foreign_movies['London']))*100

65.73770491803279

In [135]:
### 
movie_metadata['Country'].value_counts().get('United States', 0)

34117

In [ ]:
len(movie_metadata['Country'])

In [130]:
### Scaled by % of  a country divided the total foreign movies 

coi = 'London'
data_foreign = {}


for country in regrouped_by_city_and_country[coi].keys() :
    
    df = pd.DataFrame({
        'Latitude' : [country_coordinates[country][0]],
        'Longitude' : [country_coordinates[country][1]],
        'Representation of COI in %' : (len(regrouped_by_city_and_country[coi][country])/len(foreign_movies[coi]))*100
    })
    data_foreign[country] = df

In [ ]:
countries_movies['USA']

In [ ]:
### Scaled by % of  a country divided the total countries movies 

coi = 'London'
data_foreign = {}


for country in regrouped_by_city_and_country[coi].keys() :
    
    df = pd.DataFrame({
        'Latitude' : [country_coordinates[country][0]],
        'Longitude' : [country_coordinates[country][1]],
        'Representation of COI in %' : (len(regrouped_by_city_and_country[coi][country])/len(countries_movies[country]))*100
    })
    data_foreign[country] = df

In [131]:
data_foreign

{'United States':    Latitude  Longitude  Representation of COI in %
 0  37.09024 -95.712891                   65.737705,
 'France':     Latitude  Longitude  Representation of COI in %
 0  46.227638   2.213749                    8.196721,
 'Hong Kong':     Latitude   Longitude  Representation of COI in %
 0  22.319304  114.169361                    0.491803,
 'Australia':     Latitude   Longitude  Representation of COI in %
 0 -25.274398  133.775136                    2.459016,
 'Spain':     Latitude  Longitude  Representation of COI in %
 0  40.463667   -3.74922                     1.47541,
 'Russia':    Latitude   Longitude  Representation of COI in %
 0  61.52401  105.318756                    0.327869,
 'India':     Latitude  Longitude  Representation of COI in %
 0  20.593684   78.96288                   10.163934,
 'Austria':     Latitude  Longitude  Representation of COI in %
 0  47.516231  14.550072                    0.491803,
 'Ireland':     Latitude  Longitude  Representatio

In [133]:
import pandas as pd
import plotly.graph_objects as go


# Concatenate all dataframes into one
frames = [df for df in data_foreign.values()]
df = pd.concat(frames, ignore_index=True)

# Scale 'Representation of COI in %' for marker size
df['marker_size'] = df['Representation of COI in %']   # Example scaling factor

# Creating the map
fig = go.Figure(go.Scattergeo(
    lat=df['Latitude'],
    lon=df['Longitude'],
    text=df['Representation of COI in %'],  # This can be adjusted for more informative hover text
    marker=dict(
        size=df['marker_size'],
        color='blue',  # You can customize the color
        line_color='rgb(40,40,40)',
        line_width=0.5,
        sizemode='diameter'
    ),
    hoverinfo='text'
))

# Update the layout to tune the map's appearance
fig.update_layout(
    title='Representation of London across Different Countries in %',
    geo=dict(
        scope='world',  # Can be changed to 'europe', 'usa', 'asia', etc.
        projection_type='natural earth'
    )
)

fig.show()
